In [1]:
import pandas as pd
import numpy as np
import sqlite3
import logging
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import logging


In [2]:
print("TensorFlow Version:", tf.__version__)

gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))

tf.test.gpu_device_name()

print("Available GPU devices:", tf.config.list_physical_devices('GPU'))

TensorFlow Version: 2.12.0
Num GPUs Available:  0
Available GPU devices: []


In [16]:
logging.basicConfig(filename='logs.log', level=logging.DEBUG, 
                    format='%(asctime)s:%(levelname)s:%(message)s')

conn = sqlite3.connect('../data_eng/HOUSING.db')
cursor = conn.cursor()
df = pd.read_sql("select * from Realtor_Final_Merged", conn)


cursor.close()
conn.close()

df = df.select_dtypes(include=[np.number])

df.head(2)

df['target_bins'] = pd.qcut(df['median_days_on_market'], q=10, duplicates='drop')  

train_set, temp_set = train_test_split(df, test_size=0.4, stratify=df['target_bins'], random_state=42)

validation_set, test_set = train_test_split(temp_set, test_size=0.5, random_state=42)

train_set = train_set.drop(columns=['target_bins','month_date_yyyymm','year'])
validation_set = validation_set.drop(columns=['target_bins','month_date_yyyymm','year'])
test_set = test_set.drop(columns=['target_bins','month_date_yyyymm','year'])

logging.info(f"Created Train Validate and Test sets")

train_set_x = train_set.drop('median_days_on_market', axis=1)
train_set_y = train_set['median_days_on_market']


val_set_x = validation_set.drop('median_days_on_market', axis=1)
val_set_y = validation_set['median_days_on_market']

test_set_x = test_set.drop('median_days_on_market', axis=1)
test_set_y = test_set['median_days_on_market']

In [4]:
#Non Scaled data
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(train_set_x.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear')) 


model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(train_set_x, train_set_y, validation_data=(val_set_x, val_set_y), epochs=100, batch_size=32)


val_loss = model.evaluate(val_set_x, val_set_y)
test_loss = model.evaluate(test_set_x, test_set_y)

print("Validation Loss:", val_loss)
print("Test Loss:", test_loss)

Epoch 1/100


2023-12-01 00:26:23.685389: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


38034/38034 [==============================] - 17s 441us/step - loss: 634112320.0000 - val_loss: 3458393.5000
Epoch 2/100
38034/38034 [==============================] - 17s 437us/step - loss: 7567715.0000 - val_loss: 29295.1113
Epoch 3/100
38034/38034 [==============================] - 17s 438us/step - loss: 9401.8896 - val_loss: 105518.2031
Epoch 4/100
38034/38034 [==============================] - 17s 438us/step - loss: 39388.3633 - val_loss: 5636.4053
Epoch 5/100
38034/38034 [==============================] - 17s 439us/step - loss: 3642.8708 - val_loss: 19503.2031
Epoch 6/100
38034/38034 [==============================] - 17s 438us/step - loss: 8242.3799 - val_loss: 20905.1875
Epoch 7/100
38034/38034 [==============================] - 17s 442us/step - loss: 8505.1221 - val_loss: 16538.8672
Epoch 8/100
38034/38034 [==============================] - 18s 470us/step - loss: 7355.0249 - val_loss: 2991.8630
Epoch 9/100
38034/38034 [==============================] - 17s 435us/step - loss: 

38034/38034 [==============================] - 16s 433us/step - loss: 2653.6824 - val_loss: 2659.6963
Epoch 73/100
38034/38034 [==============================] - 16s 433us/step - loss: 2653.7200 - val_loss: 2658.7251
Epoch 74/100
38034/38034 [==============================] - 16s 433us/step - loss: 2653.6748 - val_loss: 2658.3679
Epoch 75/100
38034/38034 [==============================] - 16s 434us/step - loss: 2653.7217 - val_loss: 2657.6270
Epoch 76/100
38034/38034 [==============================] - 16s 433us/step - loss: 2653.6960 - val_loss: 2657.6238
Epoch 77/100
38034/38034 [==============================] - 17s 437us/step - loss: 2653.7776 - val_loss: 2657.7310
Epoch 78/100
38034/38034 [==============================] - 16s 433us/step - loss: 2653.7195 - val_loss: 2658.9128
Epoch 79/100
38034/38034 [==============================] - 16s 432us/step - loss: 2653.7808 - val_loss: 2657.5964
Epoch 80/100
38034/38034 [==============================] - 16s 434us/step - loss: 2653.7537 

In [6]:
def build_model_nonscale(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), 
                           activation='relu', input_shape=(train_set_x.shape[1],)))
    model.add(layers.Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(layers.Dense(1, activation='linear'))  # Output layer for regression

    model.compile(optimizer=keras.optimizers.Adam(
        hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error')
    
    return model

tuner = kt.RandomSearch(
    build_model_nonscale,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='ann_regression'
)

tuner.search(train_set_x, train_set_y, epochs=20, validation_data=(val_set_x, val_set_y))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
best_model = build_model_nonscale(best_hps)

# Fit the best model
history = best_model.fit(train_set_x, train_set_y, epochs=20, validation_data=(val_set_x, val_set_y))

# Evaluate on the test set
test_loss = best_model.evaluate(test_set_x, test_set_y)
print("Test Loss:", test_loss)

Reloading Tuner from my_dir/ann_regression/tuner0.json
Epoch 1/20
38034/38034 [==============================] - 16s 421us/step - loss: 1446086912.0000 - val_loss: 251366240.0000
Epoch 2/20
38034/38034 [==============================] - 16s 422us/step - loss: 15076349.0000 - val_loss: 65552.1016
Epoch 3/20
38034/38034 [==============================] - 16s 423us/step - loss: 246817.2812 - val_loss: 561289.2500
Epoch 4/20
38034/38034 [==============================] - 16s 422us/step - loss: 156292.7031 - val_loss: 404751.4375
Epoch 5/20
38034/38034 [==============================] - 16s 423us/step - loss: 152602.6562 - val_loss: 391499.8438
Epoch 6/20
38034/38034 [==============================] - 16s 424us/step - loss: 162702.3594 - val_loss: 554630.6875
Epoch 7/20
38034/38034 [==============================] - 16s 423us/step - loss: 284626.5625 - val_loss: 70476.4922
Epoch 8/20
38034/38034 [==============================] - 16s 421us/step - loss: 3443.7158 - val_loss: 530067.3125
Epoc

In [7]:
pred_nonscale = best_model.predict(test_set_x)

mse = mean_squared_error(test_set_y, pred_nonscale)
print("Mean Square Error for vanilla ANN Model - Non Scaled data")
print(mse)

12678/12678 [==============================] - 3s 236us/step
Mean Square Error for vanilla ANN Model - Non Scaled data
59019.77014303767


In [18]:
#scale all the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_set_x)
X_val_scaled = scaler.transform(val_set_x)
X_test_scaled = scaler.transform(test_set_x)

#train_set_y = train_set_y.reset_index()
#val_set_y = val_set_y.reset_index()
#test_set_y = test_set_y.reset_index()

train_set_y = train_set_y.reshape(-1, 1)
val_set_y = val_set_y.reshape(-1, 1)
test_set_y = test_set_y.reshape(-1, 1)

Y_train_scaled = scaler.fit_transform(train_set_y)
Y_val_scaled = scaler.transform(val_set_y)
Y_test_scaled = scaler.transform(test_set_y)

Y_train_scaled

TypeError: 'module' object is not callable

In [8]:
#instantiate model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear')) 


model.compile(optimizer='adam', loss='mean_squared_error')
#fit data 
history = model.fit(X_train_scaled, Y_train_scaled, validation_data=(X_val_scaled, Y_val_scaled), epochs=100, batch_size=32)

#evaluate using MSE as loss function
val_loss = model.evaluate(X_val_scaled, Y_val_scaled)
test_loss = model.evaluate(X_test_scaled, Y_test_scaled)

print("Validation Loss:", val_loss)
print("Test Loss:", test_loss)


Epoch 1/100
38034/38034 [==============================] - 20s 516us/step - loss: 0.6037 - val_loss: 0.6025
Epoch 2/100
11679/38034 [========>.....................] - ETA: 11s - loss: 0.5945

KeyboardInterrupt: 

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), 
                           activation='relu', input_shape=(X_train_scaled.shape[1],)))
    model.add(layers.Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(layers.Dense(1, activation='linear'))  # Output layer for regression

    model.compile(optimizer=keras.optimizers.Adam(
        hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error')
    
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='ann_regression'
)

tuner.search(X_train_scaled, Y_train_scaled, epochs=20, validation_data=(X_val_scaled, Y_val_scaled))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
best_model = build_model(best_hps)

# Fit the best model
history = best_model.fit(X_train_scaled, Y_train_scaled, epochs=20, validation_data=(X_val_scaled, Y_val_scaled))

# Evaluate on the test set
test_loss = best_model.evaluate(X_test_scaled, Y_test_scaled)
print("Test Loss:", test_loss)

In [ ]:

pred = best_model.predict(X_test_scaled)

mse = mean_squared_error(test_set_y, pred)
print("Mean Square Error for vanilla ANN Model - scaled data")
print(mse)

In [ ]:
#best_model.save('ann_model')

In [ ]:
#ann_mse=mse 
#rf_mse=1808.0038216640519
#reg_mse=2261.9921642924583

In [ ]:
#import matplotlib.pyplot as plt

#fig, ax = plt.subplots()

#MSEs = [ann_mse, rf_mse, reg_mse]
#model_labels = ['ANN MSE', 'Random Forest MSE', 'Regression MSE']

#ax.bar(model_labels ,MSEs, label=model_labels)

#ax.set_ylabel('MSE')
#ax.set_title('Model MSE Comparison')

#plt.show()